In [1]:
import pandas as  pd
import pymongo
import requests
import json
from datetime import datetime
import time
import calendar

In [ ]:
bike_data_csv = "data/metro-bike-share-trips-2018-q3.csv"

In [ ]:
bike_data_df = pd.read_csv(bike_data_csv)

In [ ]:
bike_data_df['weekday'] = bike_data_df.apply(lambda row: calendar.day_name[datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').weekday()] , axis=1)

In [ ]:
bike_data_df['start_day'] = bike_data_df.apply(lambda row: datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') , axis=1)

In [ ]:
bike_data_df.head()                           

In [ ]:
bike_data_df['start_time'] = pd.to_datetime(bike_data_df['start_time'], format='%Y-%m-%d %H:%M:%S')
bike_data_df['end_time'] = pd.to_datetime(bike_data_df['end_time'], format='%Y-%m-%d %H:%M:%S')  

In [ ]:
#items = bike_data_df[:1000].to_json(orient='records', date_format='iso', date_unit='s')
items = bike_data_df[:1000].to_json(orient='records')

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017/bike_data_db'
client = pymongo.MongoClient(conn)

In [10]:
db = client.bike_data_db
collection = db.bike_trip

In [11]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bike_data_db'), 'bike_trip')

In [ ]:
items_db = json.loads(items)

In [ ]:
collection.drop()

In [ ]:
db.bike_trip.insert_many(items_db)

In [12]:
collection.count_documents({})

93199

In [ ]:
collection.distinct("start_day")

In [8]:
collection.find_one()

TypeError: 'Collection' object is not callable. If you meant to call the 'find_one' method on a 'Database' object it is failing because no such method exists.

In [ ]:
oneWayDoc = collection.find({"trip_route_category": "One Way"})

In [ ]:
oneWayDoc[0]

In [ ]:
#temp = collection.find({"start_time": {'$gte': '2018-07-01T00:00:00Z', '$lt': '2018-07-31T23:59:59Z'}})
temp = collection.find({"start_day": {'$gte': '2018-07-01', '$lt': '2018-07-31'}})

In [ ]:
list(temp)

In [ ]:
datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")

In [ ]:
from flask import jsonify

In [ ]:
bike = list(temp)

In [ ]:
bike

In [ ]:
for x in bike:
    x.pop('_id', None)
bike[0]

In [ ]:
b_link = "http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson"

In [13]:
collection = db.la_boundary

In [14]:
collection.find()

In [ ]:
r = requests.get(b_link)

In [ ]:
r.json()

In [ ]:
collection.insert(r.json())

In [ ]:
temp = collection.find()

In [ ]:
temp = list(temp)

In [ ]:
for i in temp:
    i.pop('_id', None)

In [4]:
db = client

In [ ]:
db